## Movie Recommender

In [1]:
import sqlite3, csv, re, pickle
import pandas as pd
import numpy as np
from sklearn.decomposition import NMF
from fuzzywuzzy import process


# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set(color_codes=True)
pal = sns.color_palette("Set2", 10)
sns.set_palette(pal)

#### SQLite DB connection

In [2]:
db =sqlite3.connect('movies2.db')
cur = db.cursor()
db

#### import csv file and insert into db

In [3]:
"""
# import movies csv from movielens
db.execute("CREATE TABLE IF NOT EXISTS movies (movieId INTEGER NOT NULL PRIMARY KEY, title TEXT,genres TEXT);") 

with open('ml-latest-small/movies.csv', encoding="utf8") as csv_file:
    #uses first line in file for column headings by default
    reader = csv.DictReader(csv_file) # comma is default delimiter
    to_db = [(row['movieId'], row['title'], row['genres']) for row in reader]
    
cur.executemany("INSERT INTO movies (movieId, title,genres) VALUES (?, ?, ?);", to_db)
db.commit()
"""

'\n# import movies csv from movielens\ndb.execute("CREATE TABLE IF NOT EXISTS movies (movieId INTEGER NOT NULL PRIMARY KEY, title TEXT,genres TEXT);") \n\nwith open(\'ml-latest-small/movies.csv\', encoding="utf8") as csv_file:\n    #uses first line in file for column headings by default\n    reader = csv.DictReader(csv_file) # comma is default delimiter\n    to_db = [(row[\'movieId\'], row[\'title\'], row[\'genres\']) for row in reader]\n    \ncur.executemany("INSERT INTO movies (movieId, title,genres) VALUES (?, ?, ?);", to_db)\ndb.commit()\n'

In [4]:
"""
# import ratings csv from movielens
db.execute("CREATE TABLE IF NOT EXISTS ratings (userId INTEGER,movieId INTEGER,rating FLOAT,timestamp TIMESTAMP);") 

with open('ml-latest-small/ratings.csv', encoding="utf8") as csv_file:
    #uses first line in file for column headings by default
    reader = csv.DictReader(csv_file) # comma is default delimiter
    to_db = [(row['userId'], row['movieId'], row['rating'],row['timestamp']) for row in reader]
    
cur.executemany("INSERT INTO ratings (userId,movieId,rating,timestamp) VALUES (?, ?, ?, ?);", to_db)
db.commit()
"""

'\n# import ratings csv from movielens\ndb.execute("CREATE TABLE IF NOT EXISTS ratings (userId INTEGER,movieId INTEGER,rating FLOAT,timestamp TIMESTAMP);") \n\nwith open(\'ml-latest-small/ratings.csv\', encoding="utf8") as csv_file:\n    #uses first line in file for column headings by default\n    reader = csv.DictReader(csv_file) # comma is default delimiter\n    to_db = [(row[\'userId\'], row[\'movieId\'], row[\'rating\'],row[\'timestamp\']) for row in reader]\n    \ncur.executemany("INSERT INTO ratings (userId,movieId,rating,timestamp) VALUES (?, ?, ?, ?);", to_db)\ndb.commit()\n'

In [ ]:
"""
# import tags csv from movielens
db.execute("CREATE TABLE tags (userId INTEGER,movieId INT,tag TEXT,timestamp TIMESTAMP);") 

with open('ml-latest-small/tags.csv', encoding="utf8") as csv_file:
    #uses first line in file for column headings by default
    reader = csv.DictReader(csv_file) # comma is default delimiter
    to_db = [(row['userId'], row['movieId'], row['tag'],row['timestamp']) for row in reader]
    
cur.executemany("INSERT INTO tags (userId,movieId,tag,timestamp) VALUES (?, ?, ?, ?);", to_db)
db.commit()
"""

'\n# import tags csv from movielens\ndb.execute("CREATE TABLE tags (userId INTEGER,movieId INT,tag TEXT,timestamp TIMESTAMP);") \n\nwith open(\'ml-latest-small/tags.csv\', encoding="utf8") as csv_file:\n    #uses first line in file for column headings by default\n    reader = csv.DictReader(csv_file) # comma is default delimiter\n    to_db = [(row[\'userId\'], row[\'movieId\'], row[\'tag\'],row[\'timestamp\']) for row in reader]\n    \ncur.executemany("INSERT INTO tags (userId,movieId,tag,timestamp) VALUES (?, ?, ?, ?);", to_db)\ndb.commit()\n'

In [ ]:
"""
# import links csv from movielens
db.execute("CREATE TABLE links (movieId INTEGER,imdbId INTEGER,tmdbId INTEGER);") 

with open('ml-latest-small/links.csv', encoding="utf8") as csv_file:
    #uses first line in file for column headings by default
    reader = csv.DictReader(csv_file) # comma is default delimiter
    to_db = [ (row['movieId'], row['imdbId'],row['tmdbId']) for row in reader]
    
cur.executemany("INSERT INTO links (movieId,imdbId,tmdbId) VALUES (?, ?, ?);", to_db)
db.commit()
"""

'\n# import links csv from movielens\ndb.execute("CREATE TABLE links (movieId INTEGER,imdbId INTEGER,tmdbId INTEGER);") \n\nwith open(\'ml-latest-small/links.csv\', encoding="utf8") as csv_file:\n    #uses first line in file for column headings by default\n    reader = csv.DictReader(csv_file) # comma is default delimiter\n    to_db = [ (row[\'movieId\'], row[\'imdbId\'],row[\'tmdbId\']) for row in reader]\n    \ncur.executemany("INSERT INTO links (movieId,imdbId,tmdbId) VALUES (?, ?, ?);", to_db)\ndb.commit()\n'

In [ ]:
for row in db.execute("SELECT genres FROM movies limit 10;"):
    print(row)

('Adventure|Animation|Children|Comedy|Fantasy',)
('Adventure|Children|Fantasy',)
('Comedy|Romance',)
('Comedy|Drama|Romance',)
('Comedy',)
('Action|Crime|Thriller',)
('Comedy|Romance',)
('Adventure|Children',)
('Action',)
('Action|Adventure|Thriller',)


In [ ]:
# table content from Movies
for row in db.execute("SELECT * FROM movies ORDER BY ROWID ASC LIMIT 5;"):
    print(row)

(1, 'Toy Story (1995)', 'Adventure|Animation|Children|Comedy|Fantasy')
(2, 'Jumanji (1995)', 'Adventure|Children|Fantasy')
(3, 'Grumpier Old Men (1995)', 'Comedy|Romance')
(4, 'Waiting to Exhale (1995)', 'Comedy|Drama|Romance')
(5, 'Father of the Bride Part II (1995)', 'Comedy')


In [ ]:
drop = """
DROP TABLE movies
"""
#db.executescript(drop_tags)

#### Joining tables with SQL JOINs

In [ ]:
sql = """SELECT userId,movieId,rating
         FROM ratings
      """
cur.execute(sql)
data = cur.fetchall()

In [ ]:
sql2 = """with rating_sum as 
                (SELECT  movieId,round(avg(rating),1) as rating, count(rating) as rating_cnt 
                 FROM ratings 
                 GROUP BY movieId),
                tags_combi as 
                (SELECT t1.movieId, GROUP_CONCAT(t2.tag) as all_tags
                 FROM tags t1 LEFT JOIN  tags t2
                 ON t1.movieId = t2.movieId 
                 GROUP BY t1.movieId)
         SELECT 
              m.movieId, 
              m.title,
              m.genres,
              r.rating as rating,
              r.rating_cnt as rating_cnt,
              t.all_tags,
              l.imdbId,
              l.tmdbId

         
         FROM movies m
         LEFT JOIN rating_sum r ON (m.movieId=r.movieId)
         LEFT JOIN tags_combi t ON (m.movieId=t.movieId)
         LEFT JOIN links l ON (m.movieId=l.movieId)

"""
cur.execute(sql2)
data2 = cur.fetchall()
print(data2)

#### making a panda dataframe

In [ ]:
df_mov = pd.DataFrame(data2 ,columns=['movie_Id', 'title', 'genres','rating','rating_cnt','tags','imdb_Id','tmdb_Id'])
df_mov.head(10)

In [ ]:
def extract_year(title): 
    pattern='\(([0-9]{4})\)'
    try:
        year = re.findall(pattern, title)[0]
    except:
        year=1900
    return int(year)

In [ ]:
df_mov['year'] = df_mov['title'].apply(extract_year)

#### data exploration

In [ ]:
df_mov.describe()

In [ ]:
df_mov.hist(column='rating')

In [ ]:
df_mov['rating_cnt'].plot.hist(bins=[0, 1, 2, 3, 4, 5, 6, 8, 10, 15, 20, 30, 100], xlim=(0,50)) 

In [ ]:
df_mov.hist(column='year' ,bins=100)

#### Feature engineering


In [ ]:
df_mov.genres = pd.Categorical(df_mov.genres)
df_mov['genre_id']= df_mov.genres.astype('category').cat.codes
df_mov.head()

#### NMF model for ratings data

In [ ]:
# creating input for model from ratings table
df_rat = pd.DataFrame(data ,columns=['user_Id','movie_Id','rating'])
df_rat = df_rat.set_index(['user_Id','movie_Id'])
df_rat = df_rat.unstack(1)
df_rat = df_rat.fillna(df_rat.mean())
df_rat.shape

In [ ]:
df_org = df_rat.copy()

nmf = NMF(n_components=3)
nmf.fit(df_rat)

In [ ]:
# Pickle
binary = pickle.dumps(nmf)
open('nmf_model.bin', 'wb').write(binary)

In [ ]:
binary = open('nmf_model.bin', 'rb').read()
nmf = pickle.loads(binary)

In [ ]:
P = nmf.transform(df_rat)
Q = nmf.components_
print(P.shape, Q.shape)

In [ ]:
nR = np.dot(P, Q)
df_out = pd.DataFrame(nR )
df_out.head()

In [ ]:
### predict new movies 
process.extractOne('Star Trek ', df_mov['title'])

#### user input

In [ ]:
movie1 = input("Name two movies and how you liked them \n \n Your first movie? ")
movie2 = input("Your second movie? ")
print('\n **** thank you! I am processinging your input now ***')
movie1_c = process.extractOne( movie1, df_mov['title'])
movie2_c = process.extractOne( movie2, df_mov['title'])

print('\nYou said: ', movie1,' and ', movie2, 
      '\n I found these matching movies in my database:\n',  
      movie1_c[0],' with a ',movie1_c[1],'% matching score \n and ',
      movie2_c[0],' with a ',movie2_c[1],'% matching score')

print('\n On a scale from 1 to 5 how would you rate \n',movie1_c[0], ':')
while True:
    try:
        rating1 = float(input('rating (0-5): '))
        if rating1 < 0 or rating1 > 5:
            raise ValueError #this will send it to the print message and back to the input option
        break
    except ValueError:
        print("Invalid input. The number must be in the range of 0-5.")
        
print('\n and now: ',movie2_c[0])
while True:
    try:
        rating2 = float(input('rating (0-5): '))
        if rating2 < 0 or rating2 > 5:
            raise ValueError #this will send it to the print message and back to the input option
        break
    except ValueError:
        print("Invalid input. The number must be in the range of 0-5.")

#### adding user to old dataframe


In [ ]:
df_out.tail(3)

In [ ]:
df_out = df_out.append(pd.Series([np.nan]), ignore_index = True)
df_out.tail(1)

In [ ]:
# lookup movie id from moview df
id_returned1 = df_mov.loc[df_mov['title'] == movie1_c[0], 'movie_Id']
id_returned2 = df_mov.loc[df_mov['title'] == movie2_c[0], 'movie_Id']

In [ ]:
pd.options.mode.chained_assignment = None
df_out.iloc[-1, df_pred2.columns.get_loc(int(id_returned1))] = rating1
df_out.iloc[-1, df_pred2.columns.get_loc(int(id_returned2))] = rating2

In [ ]:
df_pred1 = df_out.tail(1)
df_out = df_out.fillna(df_out.mean())
df_pred2 = df_out.tail(1)
df_out.drop(df_out.tail(1).index,inplace=True)

In [ ]:
df_pred2

In [ ]:
df_pred1

In [ ]:
df_out.tail(2)

In [ ]:
df_pred = pd.concat([pd.DataFrame(df_pred1.T), pd.DataFrame(df_pred2.T)],
                          axis=1, 
                          ignore_index=True)

df_pred = df_pred.join(df_mov['title'])

In [ ]:
# filter movies from users 
filtered = df_pred.loc[df_pred[0].isnull()]
filtered = filtered[filtered.columns[-2:]]
filtered.sort_values(by=[1], ascending=False).head(10)